In [1]:
import transformers

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    print("Invalid device or cannot modify virtual devices once initialized.")
    pass

In [3]:
tf.__version__

&#39;2.3.0&#39;

In [4]:
train_df = pd.read_json('data/train.jsonl', lines = True)
train_df["img"] = "data/" + train_df["img"]

In [5]:
val1_df = pd.read_json('data/dev_seen.jsonl', lines = True)
val2_df = pd.read_json('data/dev_unseen.jsonl', lines = True)
val_df = pd.concat([val1_df, val2_df])
val_df["img"] = "data/" + val_df["img"]

In [6]:
img_width = 224
img_height = 224

In [7]:
tokenizer = transformers.DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [8]:
max_len = 128

### Train

In [9]:
def build_model():
    
    encoder = transformers.TFDistilBertModel.from_pretrained("distilbert-base-uncased")
    encoder.trainable = False

    input_img = tf.keras.layers.Input(
        shape = (img_width, img_height, 3), name = "image"
    )
    
    input_ids = tf.keras.layers.Input(name = "input_ids", shape = (max_len,), dtype = tf.int32)

    attention_mask = tf.keras.layers.Input(name = "attention_mask", shape = (max_len,), dtype = tf.int32)

    y = encoder(input_ids, attention_mask = attention_mask)[0]
    y = tf.keras.layers.BatchNormalization()(y)
    
    y = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences = True))(y)
    y = tf.keras.layers.BatchNormalization()(y)

    extractor = tf.keras.applications.EfficientNetB0(include_top = False, input_tensor = input_img, weights = None)
    
    extractor.trainable = True
    
    x = tf.keras.layers.BatchNormalization()(extractor.output)
    x = tf.keras.layers.GlobalMaxPooling2D()(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    y = tf.keras.layers.GlobalMaxPooling1D()(y)
    y = tf.keras.layers.BatchNormalization()(y)
    
    x = tf.keras.layers.concatenate([x, y])
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Flatten()(x)
    
    x = tfa.layers.WeightNormalization(tf.keras.layers.Dense(1024, activation = "relu"))(x)
    
    x = tf.keras.layers.Dropout(0.25)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tfa.layers.WeightNormalization(tf.keras.layers.Dense(256, activation = "relu"))(x)
    
    x = tf.keras.layers.Dropout(0.25)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    out = tfa.layers.WeightNormalization(tf.keras.layers.Dense(1, activation = 'sigmoid'))(x)

    model = tf.keras.models.Model([input_img, input_ids, attention_mask], out)

    return model

In [10]:
model = build_model()
model.compile(optimizer = tfa.optimizers.LazyAdam(0.001), loss = "binary_crossentropy", metrics = ["accuracy", tf.keras.metrics.AUC(name = 'auc')])
model.summary()

______________
block6a_expand_bn (BatchNormali (None, 14, 14, 672)  2688        block6a_expand_conv[0][0]        
__________________________________________________________________________________________________
block6a_expand_activation (Acti (None, 14, 14, 672)  0           block6a_expand_bn[0][0]          
__________________________________________________________________________________________________
block6a_dwconv_pad (ZeroPadding (None, 17, 17, 672)  0           block6a_expand_activation[0][0]  
__________________________________________________________________________________________________
block6a_dwconv (DepthwiseConv2D (None, 7, 7, 672)    16800       block6a_dwconv_pad[0][0]         
__________________________________________________________________________________________________
block6a_bn (BatchNormalization) (None, 7, 7, 672)    2688        block6a_dwconv[0][0]             
______________________________________________________________________________________________

In [11]:
def encode_single_sample(img_path, label, text):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_png(img, channels = 3)
    img = tf.image.convert_image_dtype(img, tf.int32)
    img = tf.image.resize(img, [img_height, img_width])
    text = tokenizer(text, return_tensors = 'tf', max_length = 128, padding = 'max_length')
    return {"image": img, "label": label, "text": text}

In [12]:
class HatefulMemes(tf.keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, input_img_paths, label, text):
        self.batch_size = batch_size
        self.input_img_paths = input_img_paths
        self.text = text
        self.label = label

    def __len__(self):
        return len(self.input_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_label = self.label[i : i + self.batch_size]
        batch_text = self.text[i : i + self.batch_size]
        w = np.zeros((self.batch_size,) + (img_height, img_width) + (3,), dtype = "float32")
        x = np.zeros((self.batch_size,max_len,), dtype = "int32")
        y = np.zeros((self.batch_size,max_len,), dtype = "int32")
        z = np.zeros((self.batch_size), dtype = "float32")
        
        for i,j,k,l in zip(range(self.batch_size), batch_input_img_paths, batch_label, batch_text):
            sample = encode_single_sample(j,k,l)
            w[i] = sample["image"].numpy().tolist()
            x[i] = sample["text"]['input_ids'].numpy().tolist()[0]
            y[i] = sample["text"]['attention_mask'].numpy().tolist()[0]
            z[i] = sample["label"]
        
        return [w,x,y], z

In [13]:
train_gen = HatefulMemes(8, train_df["img"].values.tolist(), train_df["label"].values.tolist(), \
                         train_df["text"].values.tolist())
val_gen = HatefulMemes(8, val_df["img"].values.tolist(), val_df["label"].values.tolist(), \
                         val_df["text"].values.tolist())

In [14]:
epochs = 50
early_stopping_patience = 10

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = "val_auc", patience = early_stopping_patience, restore_best_weights = True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_auc', factor = 0.2,
                                                 patience = 5, min_lr = 0.0001)

history = model.fit(train_gen, validation_data = val_gen, epochs = epochs, callbacks = [early_stopping,                                        reduce_lr])

Epoch 1/50
1062/1062 [==============================] - 1430s 1s/step - loss: 0.6884 - accuracy: 0.6249 - auc: 0.5996 - val_loss: 1.4405 - val_accuracy: 0.5837 - val_auc: 0.5775
Epoch 2/50
1062/1062 [==============================] - 1147s 1s/step - loss: 0.6095 - accuracy: 0.6810 - auc: 0.6777 - val_loss: 0.9954 - val_accuracy: 0.5625 - val_auc: 0.5690
Epoch 3/50
1062/1062 [==============================] - 1183s 1s/step - loss: 0.5695 - accuracy: 0.7120 - auc: 0.7321 - val_loss: 0.9360 - val_accuracy: 0.5702 - val_auc: 0.5874
Epoch 4/50
1062/1062 [==============================] - 1185s 1s/step - loss: 0.5452 - accuracy: 0.7365 - auc: 0.7636 - val_loss: 0.7975 - val_accuracy: 0.5798 - val_auc: 0.6111
Epoch 5/50
1062/1062 [==============================] - 1223s 1s/step - loss: 0.5312 - accuracy: 0.7439 - auc: 0.7786 - val_loss: 0.9543 - val_accuracy: 0.5788 - val_auc: 0.6028
Epoch 6/50
1062/1062 [==============================] - 1163s 1s/step - loss: 0.5007 - accuracy: 0.7635 - auc:

### Predict

In [15]:
train_df = pd.read_json('data/test_unseen.jsonl', lines = True)
train_df["img"] = "data/" + train_df["img"]

In [16]:
def val_encode_single_sample(img_path, text):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_png(img, channels = 3)
    img = tf.image.convert_image_dtype(img, tf.int32)
    img = tf.image.resize(img, [img_height, img_width])
    text = tokenizer(text, return_tensors = 'tf', max_length = 128, padding = 'max_length')
    return {"image": img, "text": text}

In [17]:
class ValHatefulMemes(tf.keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, input_img_paths, text):
        self.batch_size = batch_size
        self.input_img_paths = input_img_paths
        self.text = text

    def __len__(self):
        return len(self.input_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_text = self.text[i : i + self.batch_size]
        w = np.zeros((self.batch_size,) + (img_height, img_width) + (3,), dtype = "float32")
        x = np.zeros((self.batch_size,max_len,), dtype = "int32")
        y = np.zeros((self.batch_size,max_len,), dtype = "int32")
        
        for i,j,k in zip(range(self.batch_size), batch_input_img_paths, batch_text):
            sample = val_encode_single_sample(j,k)
            w[i] = sample["image"].numpy().tolist()
            x[i] = sample["text"]['input_ids'].numpy().tolist()[0]
            y[i] = sample["text"]['attention_mask'].numpy().tolist()[0]
        
        return [w,x,y]

In [18]:
pred_gen = ValHatefulMemes(1, train_df["img"].values.tolist(), train_df["text"].values.tolist())

In [19]:
preds = model.predict(pred_gen, verbose = 1)

2000/2000 [==============================] - 288s 144ms/step


In [20]:
def prob2pred(x):
    if x > 0.5:
        return 1
    else:
        return 0

In [21]:
train_df["proba"] = [i[0] for i in preds.tolist()]
train_df["label"] = train_df["proba"].apply(prob2pred)

In [22]:
train_df = train_df.drop(["img","text"], axis = 1)
train_df

,id,proba,label
0,15740,0.165844,0
1,38794,0.808912,1
2,60792,0.127603,0
3,71824,0.341913,0
4,4796,0.063573,0
...,...,...,...
1995,71352,0.941351,1
1996,2164,0.094855,0
1997,3587,0.106884,0
1998,47839,0.231437,0


In [23]:
train_df.to_csv("submission_new.csv", index = False)